In [195]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as plt
from math import floor
from sklearn.preprocessing import PolynomialFeatures


#Import cranial database 
df = pd.read_csv('../../datasources/fdb/cranial.csv')

#Drop categorical data. We'll focus on measurement data.
del df['DB']
del df['Item']
del df['ID']
del df['ContNum']
del df['FDN']
del df['Pop']
del df['PopSex']
del df['Ethnicity'] #Probably really important.
del df['BirthYear']
del df['Age'] #Probably really important.
del df['Comments']

rows, cols = df.shape 
print("Starting with", rows, "individuals and", cols, "features.")

Starting with 5342 individuals and 111 features.


In [196]:
#Consider our target col, Sex.
print("Unique entries.")
df['Sex'].value_counts(dropna=False)

Unique entries.


M      3187
F      2095
N        58
NaN       2
Name: Sex, dtype: int64

In [197]:
#Clean N and Nan entries
df = df[~(df.Sex.str.contains("N") == True)]
df = df.dropna(subset = ['Sex'])

df['Sex'].value_counts(dropna=False)

M    3187
F    2095
Name: Sex, dtype: int64

In [198]:
#Move from object to float
df['Sex'] = df['Sex'].map({'M':1, 'F':0})

In [199]:
#Shuffle rows of df
df = df.iloc[np.random.permutation(len(df))]

#Split into train and test sets
p=0.8
split_pt = floor(len(df)*p)
train, test = df.iloc[0:split_pt], df[split_pt:]

In [200]:
#Use train mean and std on test data.
tr_mean = train.mean()
tr_std  = train.std()
train = ( train - tr_mean ) / tr_std
test  = ( test - tr_mean ) / tr_std

#Clean this up.
train['Sex'] = train['Sex'].apply(np.sign)
test['Sex']  = test['Sex'].apply(np.sign)
train['Sex'] = ( train['Sex'] + 1 ) / 2
test['Sex'] = ( test['Sex'] + 1 ) / 2 

In [201]:
train.describe()

/usr/local/lib/python3.5/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Sex,GOL,NOL,BNL,BBH,XCB,XFB,WFB,ZYB,AUB,...,MAN,BABR,BANA,BAPR,UFHT,UFBR,ORBR,BIOB,INTB,MOW
count,4225.000000,4.162000e+03,3.516000e+03,4.136000e+03,4.146000e+03,4.164000e+03,3.516000e+03,2.099000e+03,4.079000e+03,4.087000e+03,...,4.040000e+02,5.120000e+02,4.860000e+02,4.610000e+02,1.698000e+03,1.269000e+03,3.620000e+02,4.870000e+02,3.320000e+02,0.0
mean,0.607574,5.514303e-16,-1.386326e-15,-4.260508e-16,-1.299920e-15,1.488829e-15,1.818795e-16,-6.761835e-16,5.504572e-16,8.849187e-16,...,-9.651246e-16,-6.938894e-18,7.310110e-17,-7.321210e-17,-8.923630e-16,1.287124e-15,-1.398513e-16,1.530147e-15,7.490661e-17,NaN
std,0.488349,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN
min,0.000000,-5.586959e+00,-3.357328e+00,-3.668699e+00,-3.664309e+00,-3.167015e+00,-3.342226e+00,-3.205874e+00,-5.386418e+00,-3.366687e+00,...,-3.314051e+00,-7.476188e+00,-2.934634e+00,-3.702169e+00,-4.680744e+00,-3.436180e+00,-2.708423e+00,-3.461576e+00,-6.078030e+00,NaN
25%,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.000000,3.456473e+00,3.543261e+00,3.439414e+00,3.162185e+00,4.306704e+00,3.892638e+00,4.879350e+00,3.883785e+00,4.851884e+00,...,2.992997e+00,2.683138e+00,5.049002e+00,2.938086e+00,4.223790e+00,3.831687e+00,3.856223e+00,3.552169e+00,2.513847e+00,NaN


In [202]:
test.describe()

/usr/local/lib/python3.5/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Sex,GOL,NOL,BNL,BBH,XCB,XFB,WFB,ZYB,AUB,...,MAN,BABR,BANA,BAPR,UFHT,UFBR,ORBR,BIOB,INTB,MOW
count,1057.000000,1038.000000,880.000000,1041.000000,1036.000000,1032.000000,879.000000,523.000000,1017.000000,1019.000000,...,114.000000,124.000000,121.000000,113.000000,417.000000,336.000000,91.000000,117.000000,81.000000,0.0
mean,0.586566,-0.014641,-0.025483,-0.014968,0.005261,-0.031949,0.031172,0.039037,-0.042204,-0.062062,...,-0.112418,-0.070622,-0.041757,0.045017,0.074844,-0.011754,-0.009577,-0.054292,0.055890,NaN
std,0.492682,0.995806,0.987172,0.997263,1.008955,0.991118,0.985553,1.005782,0.984958,0.961496,...,1.097330,0.848994,0.898191,1.008208,0.991944,1.013358,0.802805,0.980047,1.024651,NaN
min,0.000000,-2.873930,-2.873076,-2.975225,-2.845130,-2.873928,-3.034359,-3.003744,-2.796214,-2.926406,...,-4.822258,-2.396525,-2.757220,-2.312348,-3.264114,-3.436180,-2.708423,-1.877827,-6.078030,NaN
25%,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.000000,3.343430,2.695820,3.439414,2.889125,4.306704,4.662304,3.464436,3.338479,3.531042,...,2.855887,1.836528,2.565204,2.783661,3.211911,3.416380,2.697756,3.325919,2.217575,NaN


In [203]:
#This is fucked. 
#test = test.reindex_axis(sorted(test.count(), reverse=True), axis=1)
#test.reindex_axis(sorted(test.count().sort_values(ascending=False).index), axis=1)
sorted(test.count(),  #.sort_values(ascending=False))

#test = test.reindex_axis(sorted(test.count, axis=1)


Sex     1057
GOL     1038
NOL      880
BNL     1041
BBH     1036
XCB     1032
XFB      879
WFB      523
ZYB     1017
AUB     1019
ASB      876
BPL      961
NPH      929
NLH     1019
JUB      851
NLB     1025
MAB      932
MAL      439
MDH      991
MDB      147
OBH     1027
OBB     1024
DKB      985
NDS      818
WNB      831
SIS      816
ZMB      883
SSS      844
FMB      855
NAS      852
        ... 
CBA       98
BFA       98
BPA       98
FXA       98
RFA      683
RPA      683
ROA      683
BSA      661
SBA      682
SLA      682
TBA      682
GNI      216
HMF      141
TMF      145
GOG      219
CDL      206
WRB      169
XRB       57
XRH       99
MLN      112
MAN      114
BABR     124
BANA     121
BAPR     113
UFHT     417
UFBR     336
ORBR      91
BIOB     117
INTB      81
MOW        0
dtype: int64

In [204]:
# poly = PolynomialFeatures(2)
# test = poly.fit_transform(test)
# test = pd.DataFrame(test)
# train= poly.fit_transform(train)
# train= pd.DataFrame()
#test.count().sort_values(ascending=False).keys()


In [205]:
#Consider imputing with knn
#Consider removing outliers
#Mean imputation
test = test.fillna(0)
train = train.fillna(0)

In [206]:
#Write to CSV
train.to_csv("data/train_baseline.csv",index=False)
test.to_csv("data/test_baseline.csv", index=False) 